# Mass Editing/Formating Excel Files

Twice a year, I have to edit around 80 excel files to get them ready to send out to each client. Instead of spending over 4 hours going though each file one at a time, I wrote a code that will do it for me. 
This code is going to mass edit 80 Excel files the exact same way each time. Here are the steps that I need the formats to be done in:

1. create a new sheet

2. copy all the data in sheet 1 into sheet 2

### Sheet 1
3. If the "Deceased?" column is all 'no', then delete the column
3a. wherever that is a 'yes', I want entire row deleted until there are no more yeses then delete the column since it's all 'no' now.
Essentially, I don't want the rows that are Deceased and no "Deceased?" column in the final product

4. In the "Solicit Code" and "Solicit Code_1" there are three values: null, 'Do Not E-Alumnae Relations', and 'Do not contact in any way. President or VPCA only!'. If the entire column is null delete the column.
4a. Wherever it says 'Do not contact in any way. President or VPCA only!', replace any text with null in the columns in that row: Addressee	Address Line 1	Address Line 2	Address Line 3	City	State	ZIP	Country	Home	Business	Cell	Email	Business Email	Officer	
4b. Wherever it says 'Do Not E-Alumnae Relations', just replace the text in the email column of that row with null.
4c. after 4a and 4b have been ran, delete "Solicit Code" and "Solicit Code_1" columns

5. Columns "Inactive" and "Invalid Address" need a conditional formating that changes the fill (background color) of cells that contain the specific text "Yes" to red.

6. Coulmns "Major", "Major_1", "Minor," "Business Email," and "Business" need to be deleted

7. sort the entire sheet by the column "Last" a - z

8. Freeze the top row for both sheets

### Sheet 2
repeat steps 4, 4a, 4b, 4c, 5 (add "Deceased?" to the conditional formating), 6, 7, and 8

### Lastly
Rename sheet 1 to "Contact List" and sheet 2 to "Class Directory"

In [56]:
# Importing all the modules and libraries that I will need.
import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.formatting.rule import CellIsRule

import warnings
warnings.simplefilter("ignore", UserWarning)

In [57]:
folder_path = "/Users/Your_Name/Folder_Path" 

In [60]:
def process_file(file_path):
    # Load the workbook and get the first sheet
    wb = load_workbook(file_path)
    ws1 = wb.active

    # Convert sheet to DataFrame for easier processing
    data = ws1.values
    columns = next(data)[0:]
    df = pd.DataFrame(data, columns=columns)

    # Step 1: Create a new sheet and copy data for Sheet 2
    ws2 = wb.create_sheet(title="Class Directory")
    for row in dataframe_to_rows(df, index=False, header=True):
        ws2.append(row)

    # Step 2: Clean up Sheet 1 (Contact List)
    # Delete columns specific to Sheet 1 only
    columns_to_delete_sheet1 = ["Major", "Major_1", "Minor", "Solicit Code", "Solicit Code_1", "Deceased?", "Business Email", "Business"]
    df = df.drop(columns=[col for col in columns_to_delete_sheet1 if col in df.columns])

    # Final columns for Sheet 1
    final_columns_sheet1 = [
        "Nickname", "First", "Maiden", "Last", "Inactive", "Invalid Address", "Status", "Class", "Addressee",
        "Address Line 1", "Address Line 2", "Address Line 3", "City", "State", "ZIP", "Country", "Home", "Business",
        "Cell", "Email", "Business Email", "Officer"
    ]

    # Ensure only the columns that exist in the DataFrame are kept
    df = df[[col for col in final_columns_sheet1 if col in df.columns]]

    # Sort Sheet 1 by "Last" column (A-Z)
    df = df.sort_values(by="Last", ascending=True)

    # Replace data in Sheet 1 with the cleaned and sorted data
    ws1.delete_rows(2, ws1.max_row)  # Clear existing rows except headers
    for row in dataframe_to_rows(df, index=False, header=False):
        ws1.append(row)

    # Step 3: Process Sheet 2 (Class Directory)
    # Extract headers and data for Sheet 2
    data2 = ws2.values
    columns2 = next(data2)[0:]  # Extract the original headers for Sheet 2
    df2 = pd.DataFrame(data2, columns=columns2)

    # Handle the "Deceased?" column in Sheet 2
    if "Deceased?" in df2.columns and df2["Deceased?"].eq("No").all():
        df2 = df2.drop(columns=["Deceased?"])  # Delete "Deceased?" if all values are "No"

    # Handle the "Solicit Code" and "Solicit Code_1" columns in Sheet 2
    if "Solicit Code" in df2.columns and "Solicit Code_1" in df2.columns:
        if df2["Solicit Code"].isnull().all() and df2["Solicit Code_1"].isnull().all():
            df2 = df2.drop(columns=["Solicit Code", "Solicit Code_1"])
        else:
            df2.loc[df2["Solicit Code"] == "Do not contact in any way. President or VPCA only!", [
                "Addressee", "Address Line 1", "Address Line 2", "Address Line 3", "City", "State", "ZIP", "Country",
                "Home", "Business", "Cell", "Email", "Business Email", "Officer"]] = None
            df2.loc[df2["Solicit Code"] == "Do Not E-Alumnae Relations", "Email"] = None
            df2 = df2.drop(columns=["Solicit Code", "Solicit Code_1"])

    # Sort Sheet 2 by "Last" column (A-Z)
    df2 = df2.sort_values(by="Last", ascending=True)

    # Final columns for Sheet 2
    final_columns_sheet2 = [
        "Nickname", "First", "Maiden", "Last", "Inactive", "Invalid Address", "Deceased?", "Status", "Class", "Major",
        "Major_1", "Minor", "Addressee", "Address Line 1", "Address Line 2", "Address Line 3", "City", "State", "ZIP",
        "Country", "Home", "Business", "Cell", "Email", "Business Email", "Officer"
    ]

    # Ensure only the specified final columns remain in Sheet 2
    df2 = df2[[col for col in final_columns_sheet2 if col in df2.columns]]

    # Replace data in Sheet 2 with the updated DataFrame
    ws2.delete_rows(2, ws2.max_row)
    for row in dataframe_to_rows(df2, index=False, header=False):
        ws2.append(row)

    # Rename sheets
    ws1.title = "Contact List"
    ws2.title = "Class Directory"

    # Save the workbook
    wb.save(file_path)

In [61]:
# Process all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx"):
        process_file(os.path.join(folder_path, file_name))

### Now I want to try and incorperate my conditional formating

In [62]:
def apply_conditional_formatting(file_path):
    # Load the workbook and the sheets
    wb = load_workbook(file_path)
    ws1 = wb["Contact List"]
    ws2 = wb["Class Directory"]

    # Define the red fill for the conditional formatting
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    # Helper function to get the column letter by name
    def get_column_letter_by_name(ws, column_name):
        for cell in ws[1]:  # First row is the header
            if cell.value == column_name:
                return cell.column_letter
        return None

    # Apply conditional formatting to Sheet 1 (Contact List)
    for column in ["Inactive", "Invalid Address"]:
        col_letter = get_column_letter_by_name(ws1, column)
        if col_letter:
            ws1.conditional_formatting.add(
                f"{col_letter}2:{col_letter}{ws1.max_row}",
                CellIsRule(operator="equal", formula=['"Yes"'], fill=red_fill)
            )

    # Apply conditional formatting to Sheet 2 (Class Directory)
    for column in ["Inactive", "Invalid Address", "Deceased?"]:
        col_letter = get_column_letter_by_name(ws2, column)
        if col_letter:
            ws2.conditional_formatting.add(
                f"{col_letter}2:{col_letter}{ws2.max_row}",
                CellIsRule(operator="equal", formula=['"Yes"'], fill=red_fill)
            )
    # Freeze the top row for both sheets
    ws1.freeze_panes = "A2"
    ws2.freeze_panes = "A2"
    
    # Save the workbook with the conditional formatting applied
    wb.save(file_path)

# Process all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file_name)
        apply_conditional_formatting(file_path)

In [63]:
def update_headers(file_path):
    # Load the workbook and the sheet
    wb = load_workbook(file_path)
    ws1 = wb["Contact List"]

    # Define the desired headers for Sheet 1
    new_headers = [
        "Nickname", "First", "Maiden", "Last", "Inactive", "Invalid Address", "Status", "Class", "Addressee",
        "Address Line 1", "Address Line 2", "Address Line 3", "City", "State", "ZIP", "Country", "Home", "Cell",
        "Email", "Officer", "", "", "", "", "", "", "", ""  
    ]

    # Replace the headers in the first row
    for col_num, header in enumerate(new_headers, start=1):
        ws1.cell(row=1, column=col_num).value = header

    # Save the workbook with updated headers
    wb.save(file_path)

# Process all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file_name)
        update_headers(file_path)

#### Now I can do this everytime so make what would have been a ~4 hour long proess, only take 10 mins with minimal editing in the end.